In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload

# %%capture --no-display
%matplotlib inline
%pylab inline

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

from src.DDM import *
from src.IAM import *
from src.dynamic_parameters import *

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def get_llh(model, data):
    llh = 0
    
    return llh

In [213]:
def get_QMLE(RT_model, RT_data, nbins=9):
    """
    Takes reaction time distribution of model and data and calculated negative log likelihood based on the Quasi-Maximum Likelihood Estimate (QMLE).
    Args: 
        RT_model: Reaction time data from model simulation. Type: array(1D, float32)
        RT_data: Reaction time data from experimental data. Type: array(1D, float32)
        nbins: Quantized bins. Default value is 9.
        
    Return:
        llh: Negatively log likelihood of the data given model parameters
        
    """
    
    
    llh = 0
    panelty_high = 1e12
    panelty_low = 1e6
    
    quantile = np.arange(0, 1+1e-10, 1/nbins)
    nTrials = len(RT_data)
    trials_per_quantile = np.diff(quantile) * nTrials
    quantile_estimate = np.zeros(nbins+1)*np.nan
    QML = np.zeros(nbins)*np.nan

    quantile_estimate[0] = min(RT_data)
    quantile_estimate[-1] = max(RT_data)

    for bin_num in range(1,nbins):
        centre = (quantile[bin_num]*nTrials) + 0.5
        prev_edge = int(np.floor(centre))
        next_edge = int(np.ceil(centre))
        quantile_estimate[bin_num] = data[prev_edge-1] + (data[next_edge-1] - data[prev_edge-1]) * (centre - prev_edge)

    for bin_num in range(0,nbins):
        likelihood = np.mean(np.logical_and(RT_model>=quantile_estimate[bin_num], RT_model<quantile_estimate[bin_num+1])) ** trials_per_quantile[bin_num]
        if likelihood != 0:
            QML[bin_num] = -log(likelihood)
        else:     # Can't take log of zero
            QML[bin_num] = panelty_high    # Higher panelty if no model RT for given data RT quantile
    
    llh = sum(QML)
    
    if (RT_model < quantile_estimate[0]).any() or (RT_model > quantile_estimate[-1]).any():
        llh += panelty_low    # Lower panelty if model RT is out of the range from data RT
        
    return llh



In [214]:

data = np.arange(2,4,1/100)
model = np.arange(2.1,4.1,1/100)

# get_QMLE(model,data, 9)
llh = get_QMLE(model,data, 10)
llh

1000475.4058280977

In [196]:
(np.array([1,2,3]) < 2).any()

True